In [ ]:
import os ,random
import tensorflow as tf
import pandas as pd,matplotlib.pyplot as plt,pathlib
import zipfile,random,matplotlib.image as mpimg

In [ ]:
import os ,random
import tensorflow as tf
import pandas as pd,matplotlib.pyplot as plt,pathlib
import zipfile,random,matplotlib.image as mpimg

In [ ]:
! kaggle datasets download -d prithwirajmitra/covid-face-mask-detection-dataset

In [ ]:
!unzip covid-face-mask-detection-dataset.zip

In [ ]:
img=mpimg.imread('New Masks Dataset/Train/Mask/0003.jpg')
plt.imshow(img)

In [ ]:
img1=img/255.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.random.set_seed(42)
t_dgen=ImageDataGenerator(rescale=1/255.)
train_dir='New Masks Dataset/Train/'
test_dir='New Masks Dataset/Test/'
train_data=t_dgen.flow_from_directory(
    train_dir,
    batch_size=32,
    target_size=(224,224),
    class_mode="binary",
    seed=42
)
test_data=t_dgen.flow_from_directory(
    test_dir,
    seed=42,
    class_mode="binary",
    target_size=(224,224),
    batch_size=32
)

model=tf.keras.Sequential([
   tf.keras.layers.Conv2D(filters=10, 
                         kernel_size=3,
                         activation="relu", 
                         input_shape=(224, 224, 3)), 
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, 
                            padding="valid"), 
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation="sigmoid")
    
])

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])
history=model.fit(train_data,
                  epochs=5,
                  steps_per_epoch=len(train_data),
                  validation_data=test_data,
                  validation_steps=len(test_data))

In [ ]:
import pandas as pd
pd.DataFrame(history.history).plot(figsize=(10, 7));

In [ ]:
def load_and_prep_image(filename, img_shape=224):
  """
  Reads an image from filename, turns it into a tensor
  and reshapes it to (img_shape, img_shape, colour_channel).
  """
 
  img = tf.io.read_file(filename)

  
  img = tf.image.decode_image(img, channels=3)

  
  img = tf.image.resize(img, size = [img_shape, img_shape])

  
  img = img/255.
  img=tf.expand_dims(img,axis=0)
  return img

In [ ]:

import pathlib
import numpy as np
data_dir = pathlib.Path("New Masks Dataset/Train") 
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names)

In [ ]:
def pred(filename):
  img1=mpimg.imread(filename)
  plt.imshow(img1)
  img=load_and_prep_image(filename)
  pred=model.predict(img)
  m=class_names[int(tf.round(pred))]
  plt.title(m)
  plt.axis=False

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
 
  display(Image(filename))
except Exception as err:
 
  print(str(err))

In [ ]:
test='photo.jpg'
test=load_and_prep_image(test)
test.shape

In [ ]:
pred('photo.jpg')